[示例文章出处](https://www.cnblogs.com/xk-bench/p/8379180.html)

# pandas 合并



## **merge**

- `left` 与 `right`：两个不同的DataFrame
- `how`：指的是合并(连接)的方式有inner(内连接),left(左外连接),right(右外连接),outer(全外连接);默认为inner
- `on` : 指的是用于连接的列索引名称。必须存在右右两个DataFrame对象中，如果没有指定且其他参数也未指定则以两个DataFrame的列名交集做为连接键
- `left_on`：左则DataFrame中用作连接键的列名;这个参数中左右列名不相同，但代表的含义相同时非常有用。
- `right_on`：右则DataFrame中用作 连接键的列名
- `left_index`：使用左则 DataFrame 中的行索引做为连接键
- `right_index`：使用右则 DataFrame 中的行索引做为连接键
- `sort`：默认为True，将合并的数据进行排序。在大多数情况下设置为False可以提高性能
- `suffixes`：字符串值组成的元组，用于指定当左右DataFrame存在相同列名时在列名后面附加的后缀名称，默认为('_x','_y')
- `copy`：默认为True,总是将数据复制到数据结构中；大多数情况下设置为False可以提高性能
- `indicator`：在 0.17.0中还增加了一个显示合并数据中来源情况；如只来自己于左边(left_only)、两者(both)

In [2]:
from pandas import Series, DataFrame, merge
import numpy as np

In [3]:
data = DataFrame([
    {"id":0,"name":'lxh',"age":20,"cp":'lm'},
    {"id":1,"name":'xiao',"age":40,"cp":'ly'},
    {"id":2,"name":'hua',"age":4,"cp":'yry'},
    {"id":3,"name":'be',"age":70,"cp":'old'}
])

data1 = DataFrame([
    {"id":100,"name":'lxh','cs':10},
    {"id":101,"name":'xiao','cs':40},
    {"id":102,"name":'hua2','cs':50}
])

data2 = DataFrame([
    {"id":0,"name":'lxh','cs':10},
    {"id":101,"name":'xiao','cs':40},
    {"id":102,"name":'hua2','cs':50}
])

In [74]:
data

,id,name,age,cp
0,0,lxh,20,lm
1,1,xiao,40,ly
2,2,hua,4,yry
3,3,be,70,old


In [75]:
data1

,id,name,cs
0,100,lxh,10
1,101,xiao,40
2,102,hua2,50


In [76]:
data2

,id,name,cs
0,0,lxh,10
1,101,xiao,40
2,102,hua2,50


### 单个列名做为内链接的连接键

In [31]:
# 单个列名做为内链接的连接键
merge(data, data1, on="name", suffixes=('_a','_b'))

,id_a,name,age,cp,id_b,cs
0,0,lxh,20,lm,100,10
1,1,xiao,40,ly,101,40


### 多列名做为内链接的连接键

In [30]:
# 多列名做为内链接的连接键
merge(data, data2, on=("name","id"))

,id,name,age,cp,cs
0,0,lxh,20,lm,10


###  不指定 on 则以两个 DataFrame 的列名交集做为连接键

In [32]:
# 不指定 on 则以两个 DataFrame 的列名交集做为连接键
merge(data, data2)

,id,name,age,cp,cs
0,0,lxh,20,lm,10


###  行索引做为连接键

In [36]:
# 设置行索引名称
indexed_data1 = data1.set_index("name")

# 使用右边的 DataFrame 的行索引做为连接键 (right_index=True)
merge(data, indexed_data1, left_on='name', right_index=True)

,id_x,name,age,cp,id_y,cs
0,0,lxh,20,lm,100,10
1,1,xiao,40,ly,101,40


###  左连接

In [43]:
# 左外连接 1 (how="left")
merge(data, data1, on="name", how="left")

,id_x,name,age,cp,id_y,cs
0,0,lxh,20,lm,100.0,10.0
1,1,xiao,40,ly,101.0,40.0
2,2,hua,4,yry,NaN,NaN
3,3,be,70,old,NaN,NaN


In [6]:
# 左外连接 2 (how="left")
test = merge(data1, data, on="name", how="left")
test

,id_x,name,cs,id_y,age,cp
0,100,lxh,10,0.0,20.0,lm
1,101,xiao,40,1.0,40.0,ly
2,102,hua2,50,NaN,NaN,NaN


###  右连接

In [45]:
# 右外连接
merge(data, data1, on="name", how="right")

,id_x,name,age,cp,id_y,cs
0,0.0,lxh,20.0,lm,100,10
1,1.0,xiao,40.0,ly,101,40
2,NaN,hua2,NaN,NaN,102,50


###  使用 left_on 与 right_on 来指定列名字不同的连接键

In [47]:
data3 = DataFrame([
    {"mid":0,"mname":'lxh','cs':10},
    {"mid":101,"mname":'xiao','cs':40},
    {"mid":102,"mname":'hua2','cs':50}
])

data3

,mid,mname,cs
0,0,lxh,10
1,101,xiao,40
2,102,hua2,50


In [48]:
# 使用 left_on 与 right_on 来指定列名字不同的连接键
merge(data, data3, left_on=["name","id"], right_on=["mname","mid"])

,id,name,age,cp,mid,mname,cs
0,0,lxh,20,lm,0,lxh,10


## join

join 方法提供了一个简便的方法用于将两个 DataFrame 中的不同的列索引合并成为一个DataFrame。

其中参数的意义与 merge 方法基本相同,只是 join 方法默认为左外连接 how=left。

In [64]:
data = DataFrame(
    [
        {"id":0,"name":'lxh',"age":20,"cp":'lm'},
        {"id":1,"name":'xiao',"age":40,"cp":'ly'},
        {"id":2,"name":'hua',"age":4,"cp":'yry'},
        {"id":3,"name":'be',"age":70,"cp":'old'}
    ], 
    index=['a','b','c','d']
)

data1 = DataFrame(
    [
        {"sex":0},
        {"sex":1},
        {"sex":2}
    ],
    index=['a','b','e']
)

In [52]:
data

,id,name,age,cp
a,0,lxh,20,lm
b,1,xiao,40,ly
c,2,hua,4,yry
d,3,be,70,old


In [53]:
data1

,sex
a,0
b,1
e,2


### 左连接

In [54]:
# 使用默认的左连接
data.join(data1)

,id,name,age,cp,sex
a,0,lxh,20,lm,0.0
b,1,xiao,40,ly,1.0
c,2,hua,4,yry,NaN
d,3,be,70,old,NaN


### 右连接

In [55]:
# 使用右连接
data.join(data1, how="right")

,id,name,age,cp,sex
a,0.0,lxh,20.0,lm,0
b,1.0,xiao,40.0,ly,1
e,NaN,NaN,NaN,NaN,2


### 内连接

In [56]:
# 使用内连接
data.join(data1, how='inner')

,id,name,age,cp,sex
a,0,lxh,20,lm,0
b,1,xiao,40,ly,1


### 全外连接

In [57]:
# 使用全外连接
data.join(data1, how='outer')

,id,name,age,cp,sex
a,0.0,lxh,20.0,lm,0.0
b,1.0,xiao,40.0,ly,1.0
c,2.0,hua,4.0,yry,NaN
d,3.0,be,70.0,old,NaN
e,NaN,NaN,NaN,NaN,2.0


## concat

concat 方法相当于数据库中的全连接(UNION ALL), 可以指定按某个轴进行连接, 也可以指定连接的方式 join(outer,inner 只有这两种)。

与数据库不同的是 concat 不会去重，要达到去重的效果可以使用 drop_duplicates 方法

```py
concat(objs, 
       axis=0, 
       join='outer', 
       join_axes=None, 
       ignore_index=False, 
       keys=None, 
       levels=None, 
       names=None, 
       verify_integrity=False, 
       copy=True):
```

In [66]:
from pandas import Series, DataFrame, concat

df1 = DataFrame({
    'city': ['Chicago', 'San Francisco', 'New York City'], 
    'rank': range(1, 4)
})

df2 = DataFrame({
    'city': ['Chicago', 'Boston', 'Los Angeles'], 
    'rank': [1, 4, 5]
})

In [62]:
df1

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3


In [63]:
df2

,city,rank
0,Chicago,1
1,Boston,4
2,Los Angeles,5


### 按轴进行内连接

In [67]:
# 按轴进行内连接
concat([df1, df2], join="inner", axis=1)

,city,rank,city,rank
0,Chicago,1,Chicago,1
1,San Francisco,2,Boston,4
2,New York City,3,Los Angeles,5


### 外连接并指定 keys(行索引)

In [68]:
# 进行外连接并指定 keys(行索引)
concat([df1, df2], keys=['a','b'])   # 这里有重复的数据

city  rank
a 0        Chicago     1
  1  San Francisco     2
  2  New York City     3
b 0        Chicago     1
  1         Boston     4
  2    Los Angeles     5

### 去重

In [69]:
# 去重后
concat([df1,df2], ignore_index=True).drop_duplicates()

,city,rank
0,Chicago,1
1,San Francisco,2
2,New York City,3
4,Boston,4
5,Los Angeles,5
